# Data exploration

In [7]:
import pandas as pd


df = pd.read_csv("data/data.csv")
df.head()

,chain_id,first_res,input,dssp8
0,4trtA,1,VMKANVTKKTLNEGLGLLERVIPSRSSNPLLTALKVETSEGGLTLS...,CEEEEEEHHHHHHHHHHHHHHSCSCCSSTTTTEEEEEECSSEEEEE...
1,5g0mA,19,PVRVGLSVDASALGHTIPPDYTGLSYEQAQMANPNYFSGANTQLAG...,CEEEEEEEEEEEEEEEEPTTCCEEEEEGGGGGCTTTSSTTCHHHHH...
2,5l6mF,2,AYVLDTNVAIHLRDGDPEVTTRVTALNGAILLSIISRVELEGGVYR...,CEEECHHHHHHHHTTCHHHHHHHHHCCSCEEEEHHHHHHHHHHHTS...
3,1gytE,1,MEFSVKSGSPEKQRSACIVVGVFEPRRLSPIAEQLDKISDGYISAL...,CEEEEECCCGGGCCCSCEEEEEETTTEECHHHHHHHHHHTSHHHHH...
4,1h0hL,1,SKGFFVDTTRCTACRGCQVACKQWHGNPATPTENTGFHQNPPDFNF...,CEEEEEEGGGCCCCCHHHHHHHHHHTPPPCCCCCSSCSCCSSSCBT...


In [3]:
print("Number of data points: ", len(df))

Number of data points:  18731


In [8]:
secondary_structures = {"G": "310 helix", "H": "α-helix", "I": "π-helix", "E": "β-sheet", "B": "β-bridge", "T": "helix turn", "S": "bend", "C": "coil"}

df["Total secondary structures"] = df["dssp8"].apply(lambda x: len(x))

# Count the number of each secondary structure element in the dssp8 column for each row and store the counts in a new column.
for key in secondary_structures.keys():
    df[secondary_structures[key]] = df["dssp8"].apply(lambda x: x.count(key))

df.head()

,chain_id,first_res,input,dssp8,Total secondary structures,310 helix,α-helix,π-helix,β-sheet,β-bridge,helix turn,bend,coil
0,4trtA,1,VMKANVTKKTLNEGLGLLERVIPSRSSNPLLTALKVETSEGGLTLS...,CEEEEEEHHHHHHHHHHHHHHSCSCCSSTTTTEEEEEECSSEEEEE...,361,20,65,0,154,1,25,34,53
1,5g0mA,19,PVRVGLSVDASALGHTIPPDYTGLSYEQAQMANPNYFSGANTQLAG...,CEEEEEEEEEEEEEEEEPTTCCEEEEEGGGGGCTTTSSTTCHHHHH...,457,21,115,0,132,2,56,40,79
2,5l6mF,2,AYVLDTNVAIHLRDGDPEVTTRVTALNGAILLSIISRVELEGGVYR...,CEEECHHHHHHHHTTCHHHHHHHHHCCSCEEEEHHHHHHHHHHHTS...,127,4,71,0,16,1,11,7,17
3,1gytE,1,MEFSVKSGSPEKQRSACIVVGVFEPRRLSPIAEQLDKISDGYISAL...,CEEEEECCCGGGCCCSCEEEEEETTTEECHHHHHHHHHHTSHHHHH...,503,30,174,0,100,0,59,38,95
4,1h0hL,1,SKGFFVDTTRCTACRGCQVACKQWHGNPATPTENTGFHQNPPDFNF...,CEEEEEEGGGCCCCCHHHHHHHHHHTPPPCCCCCSSCSCCSSSCBT...,214,8,52,0,46,2,31,22,50


In [ ]:
# average total secondary structures, mean, etc.


# average number of each secondary structure

# averge length of input

# most common amino acids in input


# correlation between length of input and number of secondary structures
